# Lab 2: Generative Adversarial Networks

## 2. InfoGAN

In this lab, we will look at a variant of GAN called InfoGAN.

Let us load a pretrained model of InfoGAN that has been trained on MNIST dataset. We are loading only the generator.


Here, we define some functions to help us generate noise as input to the generator.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from InfoGAN import InfoGAN_Generator

model = torch.load('InfoGAN_Gen_cpu.pt')
print(model)


InfoGAN_Generator (
  (fc_in): Linear (22 -> 2048)
  (convs): ModuleList (
    (0): ConvTranspose2d(32, 1, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
    (1): ConvTranspose2d(64, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
    (2): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  )
  (BNs): ModuleList (
    (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  )
)


In [7]:
def gen_noise(n_instance, n_dim=2):
    """generate n-dim uniform random noise"""
    return torch.Tensor(np.random.uniform(low=-1.0, high=1.0,
                                          size=(n_instance, n_dim)))


def gen_conti_codes(n_instance, n_conti, mean=0, std=1):
    """generate gaussian continuous codes with specified mean and std"""
    codes = np.random.randn(n_instance, n_conti) * std + mean
    return torch.Tensor(codes)


def gen_discrete_code(n_instance, n_discrete, num_category=10):
    """generate discrete codes with n categories"""
    codes = []
    for i in range(n_discrete):
        code = np.zeros((n_instance, num_category))
        random_cate = np.random.randint(0, num_category, n_instance)
        code[range(n_instance), random_cate] = 1
        codes.append(code)

    codes = np.concatenate(codes, 1)
    return torch.Tensor(codes)

Let us generate some random noise vector and see the generated images. 

In [8]:
from torch.autograd import Variable
import numpy as np
batch_size = 64
noise_dim = 10 # size of the entangled noise vector
n_conti=2 # number of latent variables controlling the continuous property
n_discrete=1 # number of latent variables controlling the discrete property (class label of generated image)

num_category=10
noises = Variable(gen_noise(batch_size, n_dim=noise_dim))
conti_codes = Variable(gen_conti_codes(batch_size, n_conti,
                                                   0.0, 0.5))
discr_codes = Variable(gen_discrete_code(batch_size, n_discrete,
                                                     num_category))

We see that the final noise vector is a concatenation of the entangled noise vector, and the latent codes that we trained. 

Let us pass the noise vector through the generator to see the output.

In [9]:
gen_inputs = torch.cat((noises, conti_codes, discr_codes), 1)
fake_inputs = model(gen_inputs)

import torchvision
import matplotlib.pyplot as plt
%matplotlib inline
output = fake_inputs.data
output = torchvision.utils.make_grid(output)
output = output.permute(1,2,0)
plt.imshow(output.numpy())

ImportError: No module named matplotlib.pyplot

### Meaning of the discrete codes

Now, let us see what the discrete codes mean. We will initialize them in a systematic manner.

In [6]:
discr_codes = Variable(torch.eye(10))
discr_codes = discr_codes[0:8,:]
discr_codes = discr_codes.repeat(8,1)
print(discr_codes.size())

_netD (
  (main): Sequential (
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)


Now, run the previous step to see the images generated.

### Meaning of the continuous codes

Let us see what the continuous codes mean. In this model, two continuous latent variables are used. We will vary them both and see the result.